In [ ]:
BE METEO

In [ ]:
#première ligne de code
print("je suis content de faire le BE")

In [ ]:
#import du fichier csv pour Colin
data_body <- read.table("a_completer/donnees-synop-essentielles-omm.csv", # Nom du fichier à importer
                        header=TRUE,     # Utiliser la première ligne du fichier pour les noms des colonnes
                        sep=";",         # Séparateur de champs
                        dec=",",         # Séparateur décimal
                        row.names=1)

In [ ]:
#import du fichier csv pour Lise
data_body <- read.table("a_completer/donnees-synop-essentielles-omm.csv", # Nom du fichier à importer
                        header=TRUE,     # Utiliser la première ligne du fichier pour les noms des colonnes
                        sep=";",         # Séparateur de champs
                        dec=",",         # Séparateur décimal
                        row.names=1)

In [ ]:
#import du fichier csv pour Anais
data_body <- read.table("a_completer/donnees-synop-essentielles-omm.csv", # Nom du fichier à importer
                        header=TRUE,     # Utiliser la première ligne du fichier pour les noms des colonnes
                        sep=";",         # Séparateur de champs
                        dec=",",         # Séparateur décimal
                        row.names=1)

In [ ]:
#import du fichier csv pour Adrien
data_body <- read.table("a_completer/donnees-synop-essentielles-omm.csv", # Nom du fichier à importer
                        header=TRUE,     # Utiliser la première ligne du fichier pour les noms des colonnes
                        sep=";",         # Séparateur de champs
                        dec=",",         # Séparateur décimal
                        row.names=1)

In [4]:
#import du fichier csv pour Thibault
data_body <- read.csv("/mnt/c/Users/Thibault/Downloads/donnees-synop-essentielles-omm.csv", # Nom du fichier à importer
                        header=TRUE,     # Utiliser la première ligne du fichier pour les noms des colonnes
                        sep=";",         # Séparateur de champs
                        dec=",",         # Séparateur décimal
                        row.names=1)

ERROR: Error in read.table(file = file, header = header, sep = sep, quote = quote, : duplicate 'row.names' are not allowed
